In [1]:
from langchain.llms import Ollama
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
import os
import pickle
import asyncio
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Initialize model mistral
model = Ollama(model="mistral_copy")

In [3]:
pdf_path = "/home/mohamed/Documents/Mohamed/Mohamed_DIALLO_CV.pdf"

In [4]:
loader = PyMuPDFLoader(pdf_path)
doc = loader.load()

In [5]:
# Définir les séparateurs pour le découpage du texte
separators = [
        "\n\n",
        "\n",
        " ",
        ".",
        ",",
        "\u200b",  # Zero-width space
        "\uff0c",  # Fullwidth comma
        "\u3001",  # Ideographic comma
        "\uff0e",  # Fullwidth full stop
        "\u3002",  # Ideographic full stop
        "",
]

In [6]:
# Initialiser le découpeur de texte avec les paramètres spécifiés
text_splitter = RecursiveCharacterTextSplitter(
    separators=separators,
    chunk_size=300, # Taille de chaque chunk en caractères
    chunk_overlap=50, # Chevauchement entre les chunks consécutifs (réduit pour moins de chunks)
    length_function=len, # Fonction pour calculer la longueur du texte
    add_start_index=True, # Ajouter l'index de début à chaque chunk
)

In [7]:

chunks = text_splitter.split_documents(doc)
print(f"Split {len(doc)} documents into {len(chunks)} chunks.")

Split 2 documents into 13 chunks.


In [8]:
# Afficher un exemple de contenu de page et de métadonnées pour un chunk
page = chunks[0]
print(page.page_content)
print(page.metadata)

Kalaban Coura ACI, Bamako, Rue 418 
Tel : +223 62 09 21 57 
Email : mohameddiallo728@gmail.com 
MOHAMED DIALLO 
Maîtrisant des outils tels que MySQL, PostgreSQL, je suis capable de façonner et d'optimiser des bases de données
{'source': '/home/mohamed/Documents/Mohamed/Mohamed_DIALLO_CV.pdf', 'file_path': '/home/mohamed/Documents/Mohamed/Mohamed_DIALLO_CV.pdf', 'page': 0, 'total_pages': 2, 'format': 'PDF 1.7', 'title': '', 'author': 'Mohamed DIALLO', 'subject': '', 'keywords': '', 'creator': 'Microsoft® Word 2019', 'producer': 'Microsoft® Word 2019', 'creationDate': "D:20240216080534+00'00'", 'modDate': "D:20240216080534+00'00'", 'trapped': '', 'start_index': 0}


In [9]:
# Chemin vers le répertoire pour enregistrer la base de données Chroma
CHROMA_PATH = "embdb"
EMB_CACHE_PATH = "emb_cache.pkl"

In [11]:
# Récupérer la fonction d'embeddings à partir des ressources du code env
emb_model = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=emb_model)

2024-07-25 18:06:34.771390: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-25 18:06:34.958922: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-25 18:06:35.047772: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-25 18:06:35.074014: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-25 18:06:35.204122: I tensorflow/core/platform/cpu_feature_guar

In [12]:
# Charger les embeddings depuis le cache ou les générer si le cache n'existe pas
if os.path.exists(EMB_CACHE_PATH):
    with open(EMB_CACHE_PATH, 'rb') as f:
        cached_embeddings = pickle.load(f)
else:
    # Générer les embeddings pour chaque chunk
    cached_embeddings = embeddings.embed_documents([chunk.page_content for chunk in chunks])
    with open(EMB_CACHE_PATH, 'wb') as f:
        pickle.dump(cached_embeddings, f)

In [13]:
# Créer une nouvelle base de données Chroma et ajouter les documents
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embeddings)
db.add_documents(documents=chunks)

['2c9a8cb5-f7ac-48b9-88fa-2fcf0956ba59',
 '0677d336-509f-4622-83a9-d01db584e50e',
 '4a32c887-c9e0-4404-8739-b0300d8eb227',
 '8b72bade-8244-406e-8b82-d62f42e2dc4f',
 '6f9c64f3-75d7-4895-a533-4282734fa2e6',
 '6bd22030-0a67-443d-a76d-0f52e909cb00',
 '682f2d65-8226-4ad3-afd3-cbab7bdacb9c',
 '4e58b653-344e-4a0b-9665-c5fa6c5c8145',
 '15d00a45-8c03-4606-921f-afdcda6cb134',
 '0ab7e633-c14b-4b3b-906f-22be4f2f7098',
 'c6fdcd8c-358e-4e10-8a60-cff5df2d2b8d',
 '385db1b7-e4a6-4ffc-bdd6-45dc7655419d',
 '09800c90-ac29-4f8e-a847-f8750dc7df9a']

In [14]:
# Persister la base de données sur le disque
db.persist()
print(f"Enregistrement de {len(chunks)} chunks dans {CHROMA_PATH}.")

Enregistrement de 13 chunks dans embdb.


In [15]:
# Function to compress prompts
def compress_prompt(context, question):
    # Keep only the most relevant parts of the context
    compressed_context = context[:1000]  # Limiter à 1000 caractères par exemple
    return f"Question: {question}\nContext: {compressed_context}"

In [16]:
# Function to handle query processing
async def process_query(query_text):
    # Retrieve context from DB using similarity search
    results = db.similarity_search_with_relevance_scores(query_text, k=5)  # Récupérer plus de résultats pour le ranking
    results.sort(key=lambda x: x[1], reverse=True)  # Tri par score de pertinence
    top_results = results[:3]  # Prendre les 3 meilleurs résultats

    context_text = "\n\n - -\n\n".join([doc.page_content for doc, _score in top_results])

    # Compress the prompt
    compressed_prompt = compress_prompt(context_text, query_text)

    # Create prompt using compressed context and query text
    prompt_template = ChatPromptTemplate.from_template("""
    Tu t'appelles Okka.
    Tu es un assistant en intelligence artificielle conçu pour aider les utilisateurs en récupérant des informations pertinentes et en générant des réponses basées sur ces informations.
    Ton objectif est de fournir des réponses claires et utiles.

    {compressed_prompt}
    """)

    # Generate response based on the prompt
    response_text = model.invoke(prompt_template.format(compressed_prompt=compressed_prompt))

    # Get sources of the relevant documents
    sources = [doc.metadata.get("source", None) for doc, _score in top_results]

    # Format and return the response
    formatted_response = f"Response: {response_text}\nSources: {sources}"
    print(formatted_response)
    return formatted_response


In [17]:
query_text = "Quelles competences Mohamed possède t'il ?"

In [27]:
# Retrieve context from DB using similarity search
results = db.similarity_search_with_relevance_scores(query_text, k=5)  # Récupérer plus de résultats pour le ranking
results.sort(key=lambda x: x[1], reverse=True)  # Tri par score de pertinence
top_results = results[:3]  # Prendre les 3 meilleurs résultats

context_text = "\n\n - -\n\n".join([doc.page_content for doc, _score in top_results])

# Compress the prompt
compressed_prompt = compress_prompt(context_text, query_text)

# Create prompt using compressed context and query text
prompt_template = ChatPromptTemplate.from_template("""
    Tu t'appelles Okka.
    Tu es un assistant en intelligence artificielle conçu pour aider les utilisateurs en récupérant des informations pertinentes et en générant des réponses basées sur ces informations.
    Ton objectif est de fournir des réponses claires et utiles.

    {compressed_prompt}
    """)

# Generate response based on the prompt
response_text = model.invoke(prompt_template.format(compressed_prompt=compressed_prompt))

# Get sources of the relevant documents
sources = [doc.metadata.get("source", None) for doc, _score in top_results]

# Format and return the response
formatted_response = f"Response: {response_text}\nSources: {sources}"
print(formatted_response)

Response:  Mohamed possède des compétences en développement backend. Durant son dernier stage de 11 mois, il a effectué la rétro-ingénierie d'applications d'administration de gestion de courses, chauffeurs, clients et recouvrements. Il est expérimenté dans une gamme diversifiée de langages de programmation allant du PHP au Python en passant par le Java.
Sources: ['/home/mohamed/Documents/Mohamed/Mohamed_DIALLO_CV.pdf', '/home/mohamed/Documents/Mohamed/Mohamed_DIALLO_CV.pdf', '/home/mohamed/Documents/Mohamed/Mohamed_DIALLO_CV.pdf']
